In [20]:
# !pip install python-aiconfig

import openai

# Use your OpenAI Key
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
# Save the original completion function
original_create = openai.chat.completions.create

In [22]:
import pprint

def pretty_print_file(file_name):
    with open(file_name, 'r') as f:
        pp = pprint.PrettyPrinter(indent=4, width = 150)
        pp.pprint(f.read())

In [23]:
import openai
from aiconfig.ChatCompletion import create_and_save_to_config

# Set Once and every call will create an AIConfig
aiconfig_settings = {
    "name": "Wow, I have such a cool name!",
    "description": "And an even cooler description!",
    "metadata": {"gpt-3.5-turbo": {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "temperature": 1
        }
    }
}
openai.chat.completions.create = create_and_save_to_config(aiconfig_settings=aiconfig_settings)

# Basic Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": False,
            "messages": [
                {
                    "content": "Hi",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
print("Chat Completion Response: ")
pprint.pprint(response)

# `aiconfig.json`` is the default file path which stores our AIConfig
# See next cell below for example of setting it ourselves
print("\nGenerated AIConfig Json: ")
pretty_print_file('aiconfig.json') 

Chat Completion Response: 
ChatCompletion(id='chatcmpl-8Q7IE1LMsEJeBOaHJVdJdn719HVgM', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1701235254, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17))

Generated AIConfig Json: 
('{\n'
 '  "name": "Wow, I have such a cool name!",\n'
 '  "schema_version": "latest",\n'
 '  "metadata": {\n'
 '    "parameters": {},\n'
 '    "models": {},\n'
 '    "gpt-3.5-turbo": {\n'
 '      "model": "gpt-3.5-turbo",\n'
 '      "top_p": 1,\n'
 '      "temperature": 1\n'
 '    }\n'
 '  },\n'
 '  "description": "And an even cooler description!",\n'
 '  "prompts": [\n'
 '    {\n'
 '      "name": "prompt_0",\n'
 '      "input": "Hi",\n'
 '      "metadata": {\n'
 '        "model": {\n'
 '          "name": "gpt-3.5-turbo",

In [24]:
import openai
from aiconfig.ChatCompletion import create_and_save_to_config

# Set Once and every call will create an AIConfig, this time setting the file path
storytelling_aiconfig_file_path = "storytelling.aiconfig.json"
aiconfig_settings = {
    "name": "AIConfig for my storytellinng prompt",
    "description": "It tells a story as specified by the prompt",
    "metadata": {"gpt-3.5-turbo": {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "temperature": 1
        }
    }
}
openai.chat.completions.create = create_and_save_to_config(config_file_path=storytelling_aiconfig_file_path, aiconfig_settings=aiconfig_settings)

# Basic Streaming Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": True,
            "messages": [
                {
                    "content": "Tell me a riveting story",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Creates a config saved to config_file_path = "storytelling.aiconfig.json"
print("Chat Completion Streaming Response: ")
for iteration in response:
    iteration_dict = iteration.model_dump()
    chunk = iteration_dict.get("choices",[{}])[0].get('delta',{}).get('content','')
    print(chunk, end = '')


print("\nGenerated AIConfig Json: ")
pretty_print_file(storytelling_aiconfig_file_path)
from openai import *

Chat Completion Streaming Response: 
Once upon a time, in a small coastal town, there lived a young girl named Emma. Emma had always been fascinated by the sea and was known for her adventurous spirit. She spent her days exploring the rocky shorelines, collecting seashells, and daydreaming about the mysteries that lay below the waves.

One sunny afternoon, Emma stumbled upon an old leather-bound book half-buried in the sand. Intrigued, she carefully picked it up and brushed away the grains. The book was filled with nautical charts, sketches of sea creatures, and tales of lost treasures.

Emma's curiosity got the better of her, and she decided to dive into the world of sea exploration. She sought advice from a wise old sailor in town, Captain James, who had sailed the deepest oceans. Captain James listened intently to Emma's passion and saw the spark in her eyes. He decided to mentor her on the art of navigation and taught her everything he knew about the sea.

With Captain James' guida

In [25]:

from aiconfig.Config import AIConfigRuntime
import openai
from aiconfig.ChatCompletion import create_and_save_to_config

# Load an existing AIConfig
storytelling_aiconfig_file_path = "storytelling.aiconfig.json"
aiconfig = AIConfigRuntime.load(storytelling_aiconfig_file_path) # load the second aiconfig we just created and use it
openai.chat.completions.create = create_and_save_to_config(aiconfig=aiconfig)

# Compounded Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "messages": [
                {
                    "content": "Tell me a joke about apples",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Config was previously loaded from 'my-second-aiconfig.json. New Prompt gets added to the AIConfig Object and also saved to the same file

completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "messages": [
                {
                    "content": "Tell me a joke about apples",
                    "role": "user",
                },
                 {
        "role": "assistant",
        "content": "Why did the apple go to school?\n\nBecause it wanted to be a \"smarty-pie\"!"
      },                                
      {
                    "content": "Tell this joke in a shakespearean rhetoric",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Config is updated with the second prompt keeping the history intact

print("\nGenerated AIConfig Json: ")
pretty_print_file(storytelling_aiconfig_file_path) # open the config yourself and check it out!


Generated AIConfig Json: 
('{\n'
 '  "name": "AIConfig for my storytellinng prompt",\n'
 '  "schema_version": "latest",\n'
 '  "metadata": {\n'
 '    "parameters": {},\n'
 '    "models": {},\n'
 '    "gpt-3.5-turbo": {\n'
 '      "model": "gpt-3.5-turbo",\n'
 '      "top_p": 1,\n'
 '      "temperature": 1\n'
 '    }\n'
 '  },\n'
 '  "description": "It tells a story as specified by the prompt",\n'
 '  "prompts": [\n'
 '    {\n'
 '      "name": "prompt_0",\n'
 '      "input": "Tell me a riveting story",\n'
 '      "metadata": {\n'
 '        "model": {\n'
 '          "name": "gpt-3.5-turbo",\n'
 '          "settings": {\n'
 '            "model": "gpt-3.5-turbo",\n'
 '            "top_p": 1,\n'
 '            "max_tokens": 3000,\n'
 '            "temperature": 1,\n'
 '            "stream": true\n'
 '          }\n'
 '        },\n'
 '        "parameters": {},\n'
 '        "remember_chat_context": true\n'
 '      },\n'
 '      "outputs": [\n'
 '        {\n'
 '          "output_type": "execute

In [26]:
# Function Call Capture
from aiconfig.Config import AIConfigRuntime
import openai
from aiconfig.ChatCompletion import create_and_save_to_config

# Set aiconfig
function_call_filepath = "function_call.aiconfig.json"
aiconfig_settings = {
    "name": "AIConfig for my weather function",
    "description": "It tells the weather for a given location"
}
openai.chat.completions.create = create_and_save_to_config(config_file_path=function_call_filepath, aiconfig_settings=aiconfig_settings)

def get_current_weather(location, unit):
    return { "temperature": 22, "unit": "celsius", "description": "Sunny" }

completion_params = {
    "model": "gpt-3.5-turbo-0613",
    "messages": [{"role": "user", "content": "What is the weather like in Boston?"}],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ],
}

response = openai.chat.completions.create(**completion_params) 

function_call_response = get_current_weather(location="Boston", unit="celsius")
print(response)

completion_params = {
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"},
    {"role": "assistant", "content": 'null', "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }},
    {"role": "function", "name": "get_current_weather", "content": str(function_call_response)}

  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}

response = openai.chat.completions.create(**completion_params) 
print(response)

ChatCompletion(id='chatcmpl-8Q7JBIW411VSvL9wINadfcRpwK0rf', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1701235313, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=82, total_tokens=100))
ChatCompletion(id='chatcmpl-8Q7JC5m36VdAoL0FIlEpxThSbFG8y', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in Boston is sunny with a temperature of 22 degrees Celsius.', role='assistant', function_call=None, tool_calls=None))], created=1701235314, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=134, total_tokens=151))


In [27]:
import openai

# Instantiate a client
from aiconfig.ChatCompletion import create_and_save_to_config

client = openai.Client()
client.chat.completions.create = create_and_save_to_config()

completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": False,
            "messages": [
                {
                    "content": "Hi there!",
                    "role": "user",
                }
            ],
        }
response = client.chat.completions.create(**completion_params)


In [28]:
from aiconfig import AIConfigRuntime
from aiconfig import InferenceOptions

config = AIConfigRuntime.load("aiconfig.json")
inference_options = InferenceOptions(stream=False)
await config.run("prompt_0", options=inference_options)

[ExecuteResult(output_type='execute_result', execution_count=0, data={'content': 'Hello! How can I help you today?', 'role': 'assistant'}, mime_type=None, metadata={'id': 'chatcmpl-8Q7JXjM0nppiBauLFgcgnfwAjfUya', 'created': 1701235335, 'model': 'gpt-3.5-turbo-0613', 'object': 'chat.completion', 'usage': {'completion_tokens': 9, 'prompt_tokens': 23, 'total_tokens': 32}, 'finish_reason': 'stop'})]